In [1]:
# !pip install -U keras-tuner # for performing hyperparameter tuning
# !pip install --upgrade tensorflow jax jaxlib

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Dense
import keras_tuner as kf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.5.3 is installed, but it is not compatible with the installed jaxlib version 0.8.0, so it will not be used.
  warnings.warn(


In [3]:
df=pd.read_csv("/content/Alphabets_data.csv")

In [4]:
df

,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,D,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,C,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,T,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,S,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [5]:
df.shape

(20000, 17)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   letter  20000 non-null  object
 1   xbox    20000 non-null  int64 
 2   ybox    20000 non-null  int64 
 3   width   20000 non-null  int64 
 4   height  20000 non-null  int64 
 5   onpix   20000 non-null  int64 
 6   xbar    20000 non-null  int64 
 7   ybar    20000 non-null  int64 
 8   x2bar   20000 non-null  int64 
 9   y2bar   20000 non-null  int64 
 10  xybar   20000 non-null  int64 
 11  x2ybar  20000 non-null  int64 
 12  xy2bar  20000 non-null  int64 
 13  xedge   20000 non-null  int64 
 14  xedgey  20000 non-null  int64 
 15  yedge   20000 non-null  int64 
 16  yedgex  20000 non-null  int64 
dtypes: int64(16), object(1)
memory usage: 2.6+ MB


In [7]:
df.describe()

,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
count,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000
mean,4.023550,7.035500,5.121850,5.37245,3.505850,6.897600,7.500450,4.628600,5.178650,8.282050,6.45400,7.929000,3.046100,8.338850,3.691750,7.80120
std,1.913212,3.304555,2.014573,2.26139,2.190458,2.026035,2.325354,2.699968,2.380823,2.488475,2.63107,2.080619,2.332541,1.546722,2.567073,1.61747
min,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,3.000000,5.000000,4.000000,4.00000,2.000000,6.000000,6.000000,3.000000,4.000000,7.000000,5.00000,7.000000,1.000000,8.000000,2.000000,7.00000
50%,4.000000,7.000000,5.000000,6.00000,3.000000,7.000000,7.000000,4.000000,5.000000,8.000000,6.00000,8.000000,3.000000,8.000000,3.000000,8.00000
75%,5.000000,9.000000,6.000000,7.00000,5.000000,8.000000,9.000000,6.000000,7.000000,10.000000,8.00000,9.000000,4.000000,9.000000,5.000000,9.00000
max,15.000000,15.000000,15.000000,15.00000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.00000,15.000000,15.000000,15.000000,15.000000,15.00000


In [8]:
df.duplicated().sum()

np.int64(1332)

In [9]:
duplicates = df[df.duplicated()]
print(duplicates)

      letter  xbox  ybox  width  height  onpix  xbar  ybar  x2bar  y2bar  \
627        I     3     7      4       5      2     7     7      0      7   
724        L     3     6      3       4      1     1     0      6      6   
730        I     1     9      0       6      0     7     7      4      4   
910        X     5    11      6       8      2     7     7      5      4   
943        O     2     3      2       1      1     8     7      6      4   
...      ...   ...   ...    ...     ...    ...   ...   ...    ...    ...   
19948      T     4     5      4       4      2     6    11      2      7   
19950      O     4     5      5       4      3     7     7      8      5   
19975      E     1     0      1       0      0     5     8      5      7   
19987      L     3     7      3       5      1     0     1      6      6   
19998      S     2     3      4       2      1     8     7      2      6   

       xybar  x2ybar  xy2bar  xedge  xedgey  yedge  yedgex  
627       13       6      

In [10]:
df

,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,D,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,C,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,T,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,S,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [11]:
df.shape

(20000, 17)

In [12]:
x=df.drop(columns=['letter'])
y=df['letter']

In [13]:
x

,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [14]:
y

,letter
0,T
1,I
2,D
3,N
4,G
...,...
19995,D
19996,C
19997,T
19998,S


In [15]:
sc=StandardScaler()
x=sc.fit_transform(x)

In [16]:
from sklearn.preprocessing import LabelEncoder
encode=LabelEncoder()
y=encode.fit_transform(y)


In [17]:
y

array([19,  8,  3, ..., 19, 18,  0])

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.20,random_state=1)

In [20]:
xtrain

array([[-0.53500387,  0.29187713, -0.55688123, ..., -0.21908163,
         0.89919847,  0.12291107],
       [ 0.51038497,  1.19973756,  0.93230511, ..., -0.8656262 ,
         1.67831553,  0.12291107],
       [ 1.03307939,  0.89711742,  1.925096  , ..., -0.21908163,
         0.89919847, -0.49535386],
       ...,
       [ 1.55577381,  0.59449727,  0.43590966, ..., -0.21908163,
        -0.26947711, -2.35014863],
       [-0.53500387, -1.82646388, -0.55688123, ..., -2.15871536,
        -0.65903564,  0.12291107],
       [-0.01230945,  0.29187713, -0.06048579, ...,  1.07400752,
        -0.26947711, -1.73188371]])

In [21]:
ytrain

array([ 4, 25, 25, ..., 19, 12,  5])

In [22]:
xtest

array([[-0.01230945, -0.31336316, -0.55688123, ..., -0.21908163,
        -0.26947711,  0.12291107],
       [-1.0576983 , -1.22122359, -1.54967212, ..., -0.21908163,
        -0.65903564, -0.49535386],
       [-0.53500387, -0.91860345, -0.55688123, ...,  1.7205521 ,
        -1.04859417,  0.12291107],
       ...,
       [ 4.16924591,  2.10759799,  2.42149144, ..., -0.8656262 ,
        -0.65903564, -0.49535386],
       [-0.01230945,  0.89711742,  0.93230511, ..., -0.8656262 ,
        -0.26947711,  0.74117599],
       [-1.0576983 , -0.6159833 , -0.55688123, ..., -2.15871536,
        -0.65903564,  0.12291107]])

In [23]:
ytest

array([ 3,  3, 21, ..., 20,  0,  0])

In [45]:
model=Sequential()

model.add(Dense(units=20, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='adadelta',loss='binary_crossentropy', metrics=['accuracy'])
model.fit(xtrain,ytrain,epochs=25)

Epoch 1/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0516 - loss: 3.6661
Epoch 2/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0489 - loss: 3.4138
Epoch 3/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0577 - loss: 3.2004
Epoch 4/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0522 - loss: 2.9558
Epoch 5/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0488 - loss: 2.8728
Epoch 6/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0535 - loss: 2.6507
Epoch 7/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0547 - loss: 2.3781
Epoch 8/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0541 - loss: 2.1673
Epoch 9/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0533 - loss: 1.8895
Epoch 10/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0560 - loss: 1.5827
Epoch 11/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0531 - loss: 1.4524
Epoch 12/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

In [39]:
ypred=model.predict(xtest)
ypred=ypred>0.5
ypred

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


array([[ True],
       [ True],
       [ True],
       ...,
       [ True],
       [ True],
       [ True]])

In [40]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       162
           1       0.04      1.00      0.07       152
           2       0.00      0.00      0.00       144
           3       0.00      0.00      0.00       156
           4       0.00      0.00      0.00       139
           5       0.00      0.00      0.00       144
           6       0.00      0.00      0.00       154
           7       0.00      0.00      0.00       146
           8       0.00      0.00      0.00       138
           9       0.00      0.00      0.00       141
          10       0.00      0.00      0.00       137
          11       0.00      0.00      0.00       139
          12       0.00      0.00      0.00       145
          13       0.00      0.00      0.00       161
          14       0.00      0.00      0.00       173
          15       0.00      0.00      0.00       174
          16       0.00      0.00      0.00       159
          17       0.00    

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [46]:
model.evaluate(xtrain,ytrain)

500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0549 - loss: -2.2624


[-2.238879919052124, 0.05256250128149986]

In [28]:
def hyper(hp):
    model = Sequential()

    # Add input layer
    # This is not just an input layer — it's the first real computational layer that:
    # Accepts 13 input features (input_dim=13), Applies a linear transformation (i.e., Wx + b)
    # Then applies a non-linear activation function (relu, tanh, or sigmoid)
    # In practice, combining the input shape into the first Dense layer is simpler and common — and again, it is a hidden layer.

    model.add(Dense(hp.Int('units0', min_value=8, max_value=128, step=8),
                    activation=hp.Choice('activation0', values=['relu', 'tanh', 'sigmoid']),
                    input_dim=16))

    # Add additional hidden layers
    for i in range(1, hp.Int('num_layers', min_value=1, max_value=10)):
        model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                        activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid'])))

    # Add output layer
    model.add(Dense(units=1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

In [29]:
tuner = kf.RandomSearch(hyper,
                        objective = 'val_accuracy',
                        max_trials = 3)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [30]:
tuner.search(xtrain,ytrain,epochs = 3, validation_data = (xtest,ytest))

Trial 3 Complete [00h 00m 09s]
val_accuracy: 0.03799999877810478

Best val_accuracy So Far: 0.04050000011920929
Total elapsed time: 00h 00m 27s


In [31]:
tuner.get_best_hyperparameters()[0].values

{'units0': 88,
 'activation0': 'relu',
 'num_layers': 10,
 'optimizer': 'sgd',
 'units1': 8,
 'activation1': 'relu',
 'units2': 8,
 'activation2': 'relu',
 'units3': 8,
 'activation3': 'relu',
 'units4': 8,
 'activation4': 'relu',
 'units5': 8,
 'activation5': 'relu',
 'units6': 8,
 'activation6': 'relu',
 'units7': 8,
 'activation7': 'relu',
 'units8': 8,
 'activation8': 'relu',
 'units9': 8,
 'activation9': 'relu'}

In [32]:
tuned_model = tuner.get_best_models(num_models=1)[0]

In [33]:
tuned_model.fit(xtrain,ytrain,epochs=50, validation_data=(xtest,ytest))

Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0414 - loss: nan - val_accuracy: 0.0405 - val_loss: nan
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0398 - loss: nan - val_accuracy: 0.0405 - val_loss: nan
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0388 - loss: nan - val_accuracy: 0.0405 - val_loss: nan
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0393 - loss: nan - val_accuracy: 0.0405 - val_loss: nan
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.0405 - loss: nan - val_accuracy: 0.0405 - val_loss: nan
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.0383 - loss: nan - val_accuracy: 0.0405 - val_loss: nan
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0400 - loss: nan - val_accuracy: 0.0405 - val_loss: nan
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0410 - loss: nan - val_accuracy: 0.0405 - val_loss: nan
Epoch 9/50
500/500 ━━━━━

In [34]:
tuned_model = tuner.get_best_models(num_models=1)[0]

In [35]:
tuned_model.fit(xtrain,ytrain,epochs=50, validation_data=(xtest,ytest))

Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0385 - loss: nan - val_accuracy: 0.0405 - val_loss: nan
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.0380 - loss: nan - val_accuracy: 0.0405 - val_loss: nan
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0404 - loss: nan - val_accuracy: 0.0405 - val_loss: nan
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0412 - loss: nan - val_accuracy: 0.0405 - val_loss: nan
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0413 - loss: nan - val_accuracy: 0.0405 - val_loss: nan
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0380 - loss: nan - val_accuracy: 0.0405 - val_loss: nan
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0398 - loss: nan - val_accuracy: 0.0405 - val_loss: nan
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0407 - loss: nan - val_accuracy: 0.0405 - val_loss: nan
Epoch 9/50
500/500 ━━━━━

In [36]:
tuned_model.evaluate(xtrain,ytrain)

500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0380 - loss: nan


[nan, 0.03918749839067459]

In [37]:
tuned_model.evaluate(xtest,ytest)

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0411 - loss: nan


[nan, 0.04050000011920929]